## Functions for generating a consistent backseries between the 2011 and 2021 ONS population estimates.

In [ ]:
#import packages
import os
import geopandas as gpd
import numpy as np
import pandas as pd
from tqdm import tqdm  # For progress bar
import folium

In [ ]:
#import datasets (user will need to change for their datasets)
os.chdir("M:\lsoa_common")

msoa_lookup = pd.read_csv("MSOA_(2011)_to_MSOA_(2021)_to_Local_Authority_District_(2022)_Lookup_for_England_and_Wales.csv")
#shapefiles
msoa_2011_shp = gpd.read_file("infuse_msoa_lyr_2011_clipped\infuse_msoa_lyr_2011_clipped.shp")
msoa_2021_shp = gpd.read_file("MOSA_2021_EW_BFC_V6_6000765062319633493\MSOA_2021_EW_BFC_V6.shp")


lsoa_lookup = pd.read_csv("LSOA_(2011)_to_LSOA_(2021)_to_Local_Authority_District_(2022)_Lookup_for_England_and_Wales_(Version_2).csv")
#shapefiles
lsoa_2011_shp = gpd.read_file("infuse_lsoa_lyr_2011\infuse_lsoa_lyr_2011.shp")
lsoa_2021_shp = gpd.read_file("Lower_layer_Super_Output_Areas_2021_EW_BFC_V8_8154990398368723939\LSOA_2021_EW_BFC_V8.shp")


## Merge and split geographies

In [ ]:
def add_population_new_based_on_chgind(df, area_code= 'LSOA'):
    """
    Depending whether the change from 2011 to 2021 area code is a merge or split deteremined by the 'CHGIND' column, 
    the function will sum the 2021 populations for splits and the sum the 2011 population for merges, 
    so that the populations will be in a equlivent area across the years

    Args:
        df (pandas.DataFrame): Input DataFrame containing population data for 2011 and 2021.
        area_code (str, optional): Code representing the area level. Default is 'LSOA'.

    Returns:
        pandas.DataFrame: DataFrame with updated population data.

    Note:
        'CHGIND' indicates whether the geography from 2011 to 2021 is being split ('S') or merged ('M') areas.

    Example:
        df = add_population_new_based_on_chgind(input_df, area_code='MSOA')
    """
    # Filter rows where CHGIND is 'S'
    filtered_df_S = df[df['CHGIND'] == 'S']
    # Group by specified columns and sum population_2021
    grouped_df_S = filtered_df_S.groupby([area_code + '11CD', 'sex_2021', 'age_2021', 'year_2021'])['population_2021'].sum().reset_index()
    # Merge the grouped data with the original DataFrame to add population_2021_new column
    merged_df_S = pd.merge(df, grouped_df_S, on=[area_code + '11CD', 'sex_2021', 'age_2021', 'year_2021'], suffixes=('', '_new'), how='left')
    # Rename the population_2021_new column
    merged_df_S.rename(columns={'population_2021_new': 'population_2021_new'}, inplace=True)
    
    # Filter rows where CHGIND is 'M'
    filtered_df_M = df[df['CHGIND'] == 'M']
    # Filter and trim unnecessary columns
    trimmed_df = filtered_df_M[[area_code + '21CD','sex_2011', 'age_2011', 'year_2011', 'population_2011']]
    # Drop duplicate rows
    trimmed_df_dropped = trimmed_df.drop_duplicates()
    # Group by specified columns and sum population_2011
    trimmed_df_dropped['population_2011'] = trimmed_df_dropped.groupby([area_code + '11CD', 'sex_2011', 'age_2011', 'year_2011'])['population_2011'].transform('sum')
    # Merge the grouped data with the original DataFrame to add population_2011_new column
    merged_df_M = pd.merge(merged_df_S, trimmed_df_dropped, on=[area_code + '11CD', 'sex_2011', 'age_2011', 'year_2011'], suffixes=('', '_new'), how='left')
    # Rename the population_2011_new column
    merged_df_M.rename(columns={'population_2011_new': 'population_2011_new'}, inplace=True)

    return merged_df_M

## Complex geoprahies

In [ ]:
def calculate_geometry_differences(shapefile_2011, shapefile_2021, lookup, area_code):
    """
    Calculate the geometry differences between two shapefiles between 2011 and 2021.

    This function merges 2011 and 2021 shapefiles, the calculates the difference in geometry between 2011 and 2021 polygons, and removes features with over 99% overlap,
    as these are not significantly different.

    Parameters:
    - shapefile_2011 (GeoDataFrame): A GeoDataFrame containing geometries for the first time period, 2011.
    - shapefile_2021 (GeoDataFrame): A GeoDataFrame containing geometries for the second time period, 2021
    - lookup (DataFrame): A DataFrame containing lookup information for the selected boundary
    - area_code (str): A string indicating the area code to use for merging and lookup (e.g., LSOA or MSOA)

    Returns:
    GeoDataFrame 1: A GeoDataFrame containing the geometry differences between the two time periods.
    GeoDataframe 2: Same geodataframe how it has filtered out rows with high overlap percentages as these difference are deem not signficantly different. 
    This geodataframe is used for mapping as it displays the boundaries that have changed significantly
    """
    # Rename geo_code to area code 11CD
    shapefile_2011 = shapefile_2011.rename(columns={'geo_code': area_code + '11CD'})

    # Filter geometry and area code 11CD
    shapefile_2011_filter = shapefile_2011[[area_code + '11CD', 'geometry']]

    # Filter shapefile_2021 based on 'area code 21CD' and 'geometry'
    shapefile_2021_filter = shapefile_2021[[area_code + '21CD', 'geometry']]

    #helpfully the column name changes slightly so we need to use a like operationso it works across dataframes
    change_column_name_str = lookup.filter(like='CH').columns[0]

    # Select 'X' from lookup
    lookup_X = lookup[lookup[change_column_name_str] == 'X']
    
    # Determine the appropriate area code suffix based on the input area_code
    code_suffix_2011 = area_code + '11CD'
    code_suffix_2021 = area_code + '21CD'
    
    # Merge lookup_X with shapefile_2011 based on the area code
    values_2011_x = pd.merge(lookup_X, shapefile_2011_filter, how='left', left_on=code_suffix_2011, right_on=area_code + '11CD')
    # Rename geometry to geometry_2011
    values_2011_x = values_2011_x.rename(columns={'geometry':'geometry_2011'})

    # Merge values_2011_x with shapefile_2021 based on the area code
    values_x = pd.merge(values_2011_x, shapefile_2021_filter, how='left', left_on=code_suffix_2021, right_on=area_code + '21CD')
    # Rename geometry to geometry_2021
    values_x = values_x.rename(columns={'geometry':'geometry_2021'})

    # Convert values_x to GeoDataFrame and reproject to EPSG:4326
    values_x = gpd.GeoDataFrame(values_x, geometry='geometry_2021')
    values_x = values_x.to_crs(epsg=4326)
    values_x = gpd.GeoDataFrame(values_x, geometry='geometry_2011')
    values_x = values_x.to_crs(epsg=4326)
    
    gdf = values_x.copy()

    # Filter out rows with missing geometries
    gdf = gdf[~gdf['geometry_2011'].isna()]
    gdf = gdf[~gdf['geometry_2021'].isna()]

    # Calculate the difference between the geometries
    difference_geometries = []
    percent_overlap = []
    for idx, row in gdf.iterrows():
        difference_geometry = row['geometry_2011'].difference(row['geometry_2021'])
        if not difference_geometry.is_empty:
            difference_geometries.append(difference_geometry)
            overlap_area = row['geometry_2011'].intersection(row['geometry_2021']).area
            percent_overlap.append(overlap_area / row['geometry_2011'].area * 100)
        else:
            percent_overlap.append(0)  # If no difference, then 100% overlap
    
    # Create GeoDataFrame for the difference polygons
    gdf_difference = gpd.GeoDataFrame(geometry=difference_geometries, crs=gdf.crs)
    gdf['percent_overlap'] = percent_overlap

    # Add the difference polygons to the original DataFrame
    gdf['difference_geometry'] = gdf_difference['geometry']

    #if percent_overlap is greater than 99 remove from df
    gdf_mapping = gdf[gdf['percent_overlap'] <= 99]
    
    return gdf, gdf_mapping

# Usage examples:
# msoa_gdf, msoa_gdf_mapping = calculate_geometry_differences(msoa_2011_shp, msoa_2021_shp, msoa_lookup, 'MSOA')
# lsoa_gdf, lsoa_gdf_mapping = calculate_geometry_differences(lsoa_2011_shp, lsoa_2021_shp, lsoa_lookup, 'LSOA')

### Map geographies layer to view and examine boundaries

In [ ]:
def create_folium_map(gdf):
    """
    Create a Folium map displaying layer for 2011 polygons, 2021 polygons, and difference between 2011 and 2021 polygons.

    Parameters:
    gdf (GeoDataFrame): A GeoDataFrame containing 'geometry_2011', 'geometry_2021', and 'difference_geometry' columns. 
    Is designed to take the geodataframe output from the function calculate_geometry_differences.

    Returns:
    folium.Map: A Folium map object displaying the polygons.
    """
    # Use set_geometry to set the geometry column
    gdf_2011 = gdf.set_geometry('geometry_2011')
    gdf_2021 = gdf.set_geometry('geometry_2021')
    gdf_difference = gdf.set_geometry('difference_geometry')

    # Convert geometry to GeoJSON format
    gdf_2011['geometry_2011_geojson'] = gdf_2011['geometry_2011'].apply(lambda x: x.__geo_interface__)
    gdf_2021['geometry_2021_geojson'] = gdf_2021['geometry_2021'].apply(lambda x: x.__geo_interface__)
    gdf_difference['difference_geometry_geojson'] = gdf_difference['difference_geometry'].apply(lambda x: x.__geo_interface__)

    # Create a Folium map centered on the UK
    map_uk = folium.Map(location=[54.7024, -3.2768], zoom_start=6)

    # Add a UK base map (you'll need to find or provide a suitable tileset)
    # For example, you can use OpenStreetMap as a generic base map
    folium.TileLayer('openstreetmap').add_to(map_uk)

    # Add GeoJSON data for 2011 polygons to the map
    geojson_2011_layer = folium.FeatureGroup(name='2011 Polygons')
    for idx, row in gdf_2011.iterrows():
        folium.GeoJson(row['geometry_2011_geojson']).add_to(geojson_2011_layer)
    map_uk.add_child(geojson_2011_layer)

    # Add GeoJSON data for 2021 polygons to the map
    geojson_2021_layer = folium.FeatureGroup(name='2021 Polygons')
    for idx, row in gdf_2021.iterrows():
        folium.GeoJson(row['geometry_2021_geojson']).add_to(geojson_2021_layer)
    map_uk.add_child(geojson_2021_layer)

    # Add GeoJSON data for difference polygons to the map
    geojson_difference_layer = folium.FeatureGroup(name='Difference Polygons')
    for idx, row in gdf_difference.iterrows():
        folium.GeoJson(row['difference_geometry_geojson']).add_to(geojson_difference_layer)
    map_uk.add_child(geojson_difference_layer)

    # Add Layer Control to toggle between different layers
    folium.LayerControl(collapsed=False).add_to(map_uk)

    return map_uk

# Usage example:
# map_uk = create_folium_map(msoa_gdf_mapping)
# map_uk.save('uk_map.html')  # Save the map as an HTML file
# map_uk  # Display the map in Jupyter Notebook or any compatible viewer

### determine and check which area codes need their populations merging 

In [ ]:
def calculate_overlapping_areas(gdf, gdf_mapping, area_code):
    """
    Calculate overlap percentages between for the polygons in gdf within the newly fromed larger polygons in gdf_mapping, this allows us to determine 
    which areas' population need to be combined such that the geopgrpahies are consistent for 2011 to 2021. this gives us the list of areacode in 2011 and 2021
    that need to be combined.

    Args:
        gdf (geopandas.GeoDataFrame): Geodataframe containing polygons.
        gdf_mapping (pandas.DataFrame): DataFrame containing mapping of polygons from 2011 to 2021.
        area_code (str): Code representing the area level.

    Returns:
        Tuple: Tuple containing:
            - Filtered DataFrame with polygons having overlap percentage greater than 90.
            - List of unique area code 11CDs.
            - List of unique area code 21CDs.

    Example:
        filtered_df, areacode11cd_list, areacode21cd_list = calculate_overlap(gdf, gdf_mapping, area_code='LSOA')
    """

    poly_2011 = gdf_mapping['geometry_2011']
    poly_2021 = gdf_mapping['geometry_2021']
    merged_polygon = poly_2011.union(poly_2021)
    gdf_mapping['new_polygon'] = merged_polygon

    overlap_columns = []

    for idx, new_polygon in tqdm(enumerate(gdf_mapping['new_polygon']), desc="Calculating overlap"):
        intersection_2011 = gpd.overlay(gdf, gpd.GeoDataFrame(geometry=[new_polygon]), how='intersection')
        intersection_2021 = gpd.overlay(gdf, gpd.GeoDataFrame(geometry=[new_polygon]), how='intersection')

        intersection_2011['intersection_area'] = intersection_2011.geometry.area
        intersection_2021['intersection_area'] = intersection_2021.geometry.area

        gdf['polygon_area_2011'] = gdf.geometry_2011.area
        gdf['polygon_area_2021'] = gdf.geometry_2021.area

        overlap_percentage_2011 = np.minimum(intersection_2011['intersection_area'] / gdf['polygon_area_2011'] * 100, 100)
        overlap_percentage_2021 = np.minimum(intersection_2021['intersection_area'] / gdf['polygon_area_2021'] * 100, 100)

        overlap_columns.append(overlap_percentage_2011)
        overlap_columns.append(overlap_percentage_2021)

    overlap_df = pd.concat(overlap_columns, axis=1)

    column_names = [f'overlap_percentage_polygon_{idx}_2011' for idx in range(len(gdf_mapping['new_polygon']))] + \
               [f'overlap_percentage_polygon_{idx}_2021' for idx in range(len(gdf_mapping['new_polygon']))]

    
    overlap_df.columns = column_names

    gdf_with_overlap = pd.concat([gdf, overlap_df], axis=1)
    
    # Filter polygons where any of the overlap percentages are greater than 90
    filtered_df = pd.DataFrame(gdf_with_overlap[gdf_with_overlap.filter(like='overlap_percentage').apply(lambda x: x > 90).any(axis=1)])

    areacode11cd_list = filtered_df[area_code + '11CD'].unique().tolist()
    #add unique LSOA21CD to list
    areacode21cd_list = filtered_df[area_code + '21CD'].unique().tolist()

    return filtered_df, areacode11cd_list, areacode21cd_list

# Usage example:
# msoa_filtered_df, msoa11cd_list, msoa21cd_list = calculate_overlap(msoa_gdf, msoa_gdf_mapping, 'MSOA')

#### to do: this need to be altered slightly so that there is a list for each new polygon

### Finally sum populations of the area codes within the new geopraghy

In [ ]:
def sum_population_to_new_geography(population_2011_data, population_2021_data, area21cd_list, area11cd_list, area_code):
    """
    Sum population data for the new geography codes for both 2011 and 2021 data, sothat the backseries is consistent

    Args:
        population_2011_data (pandas.DataFrame): Population data for 2011.
        population_2021_data (pandas.DataFrame): Population data for 2021.
        area21cd_list (list): List of unique area code 21CDs from the output of the calculate_overlapping_areas function.
        area21cd_list (list): List of unique area code 21CDs from the output of the calculate_overlapping_areas function.
        area_code (str): Code representing the area level.

    Returns:
        Tuple: Tuple containing:
            - DataFrame with summed population data for new geography codes in 2011.
            - DataFrame with summed population data for new geography codes in 2021.

    Example:
        population_2011_data_filtered_combined, population_2021_data_filtered_combined = sum_population_to_new_geography(population_2011_data, population_2021_data, msoa21cd_list, msoa11cd_list, 'MSOA')
    """
    # Sum population to new geography 2011
    population_2011_data_filtered = population_2011_data[population_2011_data[area_code + '11CD'].isin(area11cd_list)].drop(columns=['geometry', area_code + '21CD']).drop_duplicates()
    population_2011_data_filtered_combined = population_2011_data_filtered.groupby(['CHGIND', 'sex', 'age', 'year']).agg({'population': 'sum'}).reset_index()
    # Add a new geography column called 'Xa2011'
    population_2011_data_filtered_combined['new_geography_2011'] = area_code + 'x_2011'
    
    # Sum population to new geography 2021
    population_2021_data_filtered = population_2021_data[population_2021_data[area_code + '21CD'].isin(area21cd_list)].drop(columns=['geometry', area_code + '11CD']).drop_duplicates()
    population_2021_data_filtered_combined = population_2021_data_filtered.groupby(['CHGIND', 'sex', 'age', 'year']).agg({'population': 'sum'}).reset_index()
    # Add a new geography column called 'Xa2021'
    population_2021_data_filtered_combined['new_geography_2021'] = area_code + 'x_2021'
    
    return population_2011_data_filtered_combined, population_2021_data_filtered_combined
